# Lab 12 — Convolutional Neural Networks


> **주제:** 합성곱 신경망 — 필터, 풀링, LeNet, VGG-style, CIFAR-10 분류

---

## 학습 목표

| # | 목표 | 예상 시간 |
|---|---|---|
| 1 | 합성곱 연산: 필터, 스트라이드, 패딩, 특성맵 |  
| 2 | CNN 구현 — LeNet-style, MLP vs CNN 비교 |  
| 3 | 더 깊은 CNN — VGG-style, BatchNorm |  
| 4 | Exercise |  

---

**데이터셋:** CIFAR-10 (PyTorch torchvision)  
- 32×32 RGB 이미지, 10개 클래스 (비행기, 자동차, 새, 고양이, 사슴, 개, 개구리, 말, 배, 트럭)  
- Train 50,000장 → 서브셋 10,000장 (CPU 학습 속도), Test 10,000장

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import seaborn as sns

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Subset

import torchvision
import torchvision.transforms as transforms

_fp = '/System/Library/Fonts/AppleGothic.ttf'
fm.fontManager.addfont(_fp)
plt.rcParams['font.family'] = fm.FontProperties(fname=_fp).get_name()
plt.rcParams['axes.unicode_minus'] = False
sns.set_theme(style='whitegrid')

torch.manual_seed(42)
np.random.seed(42)

# 디바이스 자동 선택 (MPS → CUDA → CPU)
if torch.backends.mps.is_available():
    device = torch.device('mps')
elif torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

print(f'PyTorch version: {torch.__version__}')
print(f'Using device   : {device}')

# CIFAR-10 로드
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

trainset_full = torchvision.datasets.CIFAR10('./data', train=True,  download=True, transform=transform)
testset_full  = torchvision.datasets.CIFAR10('./data', train=False, download=True, transform=transform)

# CPU 학습을 위해 train 서브셋 10,000장 사용
torch.manual_seed(42)
train_idx    = torch.randperm(len(trainset_full))[:10000].tolist()
train_subset = Subset(trainset_full, train_idx)

train_loader = DataLoader(train_subset, batch_size=64,  shuffle=True,  num_workers=0)
test_loader  = DataLoader(testset_full,  batch_size=128, shuffle=False, num_workers=0)

classes = ('비행기', '자동차', '새', '고양이', '사슴', '개', '개구리', '말', '배', '트럭')

print(f'\nTrain (서브셋) : {len(train_subset):,}장')
print(f'Test           : {len(testset_full):,}장')
print(f'클래스         : {classes}')

In [ ]:
# CIFAR-10 샘플 시각화
def denorm(tensor):
    """정규화 해제 (0.5 mean, 0.5 std) → [0, 1]"""
    return tensor * 0.5 + 0.5

fig, axes = plt.subplots(2, 10, figsize=(15, 3.5))
shown = {c: 0 for c in range(10)}
images_by_class = {c: [] for c in range(10)}

for img, label in trainset_full:
    if len(images_by_class[label]) < 2:
        images_by_class[label].append(img)
    if all(len(v) == 2 for v in images_by_class.values()):
        break

for c in range(10):
    for row in range(2):
        img = denorm(images_by_class[c][row]).permute(1, 2, 0).numpy()
        axes[row, c].imshow(img)
        if row == 0:
            axes[row, c].set_title(classes[c], fontsize=9)
        axes[row, c].axis('off')

plt.suptitle('CIFAR-10 샘플 (클래스별 2장, 32×32 RGB)', fontsize=11)
plt.tight_layout()
plt.show()

# 이미지 형태 확인
img, label = train_subset[0]
print(f'이미지 shape: {img.shape}  (채널, 높이, 너비)')
print(f'픽셀 범위   : [{img.min():.2f}, {img.max():.2f}]  (정규화 후)')

---
## Part 1. 합성곱 연산 (Convolution Operation)

### 1-1. 필터(커널)란?

합성곱은 **작은 필터(커널)**가 이미지 위를 슬라이딩하며 내적(dot product)을 계산하는 연산입니다.

$$\text{output}[i, j] = \sum_{m=0}^{K-1} \sum_{n=0}^{K-1} \text{input}[i \cdot S + m,\; j \cdot S + n] \times \text{kernel}[m, n]$$

| 용어 | 설명 |
|---|---|
| **필터/커널** | 학습되는 작은 가중치 행렬 (e.g., 3×3, 5×5) |
| **스트라이드 S** | 필터가 한 번에 이동하는 픽셀 수 |
| **패딩 P** | 이미지 가장자리에 추가하는 0 픽셀 |
| **특성맵** | conv 레이어의 출력 (각 필터마다 하나씩) |

**출력 크기 공식:**
$$H_{\text{out}} = \left\lfloor \frac{H_{\text{in}} - K + 2P}{S} \right\rfloor + 1$$

### 1-2. 다양한 필터 효과

수작업 필터를 통해 CNN이 학습하는 특성(edge, texture 등)을 직관적으로 이해합니다.

In [ ]:
# 수작업 필터 정의 및 시각화
import scipy.signal as signal

# CIFAR-10 이미지 그레이스케일 변환
img_tensor, lbl = trainset_full[3]
img_rgb  = denorm(img_tensor).permute(1, 2, 0).numpy()          # (32,32,3)
img_gray = img_rgb.mean(axis=2)                                   # (32,32)

# 다양한 수작업 필터
filters = {
    'Identity':           np.array([[0, 0, 0],
                                    [0, 1, 0],
                                    [0, 0, 0]], dtype=np.float32),
    'Blur (평균)':        np.ones((3, 3), dtype=np.float32) / 9,
    'Sharpen (선명화)':   np.array([[ 0,-1, 0],
                                    [-1, 5,-1],
                                    [ 0,-1, 0]], dtype=np.float32),
    'Edge (수평)':        np.array([[-1,-2,-1],
                                    [ 0, 0, 0],
                                    [ 1, 2, 1]], dtype=np.float32),
    'Edge (수직)':        np.array([[-1, 0, 1],
                                    [-2, 0, 2],
                                    [-1, 0, 1]], dtype=np.float32),
    'Emboss (양각)':      np.array([[-2,-1, 0],
                                    [-1, 1, 1],
                                    [ 0, 1, 2]], dtype=np.float32),
}

fig, axes = plt.subplots(3, len(filters) + 1, figsize=(16, 7))

# 원본
axes[0, 0].imshow(img_rgb);       axes[0, 0].set_title('원본 (RGB)', fontsize=9)
axes[1, 0].imshow(img_gray, cmap='gray'); axes[1, 0].set_title('그레이스케일', fontsize=9)
axes[2, 0].axis('off')
for row in range(3): axes[row, 0].axis('off') if row == 2 else axes[row, 0].set_xticks([])

for col, (name, kernel) in enumerate(filters.items(), start=1):
    # 필터 시각화
    im = axes[0, col].imshow(kernel, cmap='RdBu_r', vmin=-2, vmax=2)
    axes[0, col].set_title(name, fontsize=8)
    for i in range(kernel.shape[0]):
        for j in range(kernel.shape[1]):
            axes[0, col].text(j, i, f'{kernel[i,j]:.0f}', ha='center', va='center', fontsize=8)
    axes[0, col].set_xticks([]); axes[0, col].set_yticks([])

    # 필터 적용 결과 (numpy conv)
    result = signal.convolve2d(img_gray, kernel, mode='same', boundary='fill')
    axes[1, col].imshow(result, cmap='gray')
    axes[1, col].set_xticks([]); axes[1, col].set_yticks([])

    # PyTorch Conv2d로 검증
    x_t = torch.tensor(img_gray[np.newaxis, np.newaxis], dtype=torch.float32)
    k_t = torch.tensor(kernel[np.newaxis, np.newaxis], dtype=torch.float32)
    result_pt = F.conv2d(x_t, k_t, padding=1)[0, 0].numpy()
    axes[2, col].imshow(result_pt, cmap='gray')
    axes[2, col].set_xlabel('PyTorch 검증', fontsize=7)
    axes[2, col].set_xticks([]); axes[2, col].set_yticks([])

axes[0, 0].set_ylabel('필터 (3×3)', fontsize=9)
axes[1, 0].set_ylabel('적용 결과', fontsize=9)
axes[2, 0].set_ylabel('F.conv2d', fontsize=9)
axes[2, 0].set_xticks([]); axes[2, 0].set_yticks([])

plt.suptitle(f'다양한 필터 효과 — 클래스: {classes[lbl]}', fontsize=12)
plt.tight_layout()
plt.show()

print('📌 핵심: CNN은 이러한 필터를 데이터에서 자동으로 학습합니다!')

In [ ]:
# 스트라이드 & 패딩 효과: 출력 크기 변화
x = torch.randn(1, 3, 32, 32)  # (Batch, Channel, H, W)

configs = [
    ('K=3, S=1, P=0', dict(in_channels=3, out_channels=1, kernel_size=3, stride=1, padding=0)),
    ('K=3, S=1, P=1', dict(in_channels=3, out_channels=1, kernel_size=3, stride=1, padding=1)),
    ('K=3, S=2, P=0', dict(in_channels=3, out_channels=1, kernel_size=3, stride=2, padding=0)),
    ('K=3, S=2, P=1', dict(in_channels=3, out_channels=1, kernel_size=3, stride=2, padding=1)),
    ('K=5, S=1, P=0', dict(in_channels=3, out_channels=1, kernel_size=5, stride=1, padding=0)),
    ('K=5, S=1, P=2', dict(in_channels=3, out_channels=1, kernel_size=5, stride=1, padding=2)),
]

print(f'입력 크기: {x.shape}  (B=1, C=3, H=32, W=32)\n')
print(f'{"설정":<18} {"출력 크기":<12} {"공식 계산":>20}')
print('-' * 56)

for name, cfg in configs:
    conv = nn.Conv2d(**cfg)
    with torch.no_grad():
        out = conv(x)
    K = cfg['kernel_size']; S = cfg['stride']; P = cfg['padding']
    H_formula = (32 - K + 2*P) // S + 1
    print(f'{name:<18} {str(tuple(out.shape)):<12} {f"({32}-{K}+2×{P})/{S}+1 = {H_formula}":>20}')

print()
print('📌 핵심 규칙:')
print('  - P=K//2, S=1 → 출력 크기 = 입력 크기  ("same" padding)')
print('  - S=2          → 출력 크기 ≈ 입력 크기 / 2  (공간 다운샘플링)')

# 스트라이드별 특성맵 시각화
fig, axes = plt.subplots(1, 4, figsize=(14, 3.5))

img_t = denorm(img_tensor).unsqueeze(0)  # (1,3,32,32)
stride_configs = [
    ('원본 32×32',          img_t[0].mean(0).numpy()),
    ('S=1, P=0 → 30×30',   nn.Conv2d(3,1,3,stride=1,padding=0)(img_t)[0,0].detach().numpy()),
    ('S=1, P=1 → 32×32',   nn.Conv2d(3,1,3,stride=1,padding=1)(img_t)[0,0].detach().numpy()),
    ('S=2, P=1 → 16×16',   nn.Conv2d(3,1,3,stride=2,padding=1)(img_t)[0,0].detach().numpy()),
]

for ax, (title, fmap) in zip(axes, stride_configs):
    ax.imshow(fmap, cmap='viridis')
    ax.set_title(title, fontsize=9)
    ax.axis('off')

plt.suptitle('스트라이드 & 패딩에 따른 출력 크기 변화', fontsize=11)
plt.tight_layout()
plt.show()

In [ ]:
# 수용 영역 (Receptive Field) 개념 시각화
# 레이어를 쌓을수록 각 출력 뉴런이 '보는' 입력 영역이 넓어집니다

fig, axes = plt.subplots(1, 4, figsize=(14, 4))

def draw_receptive(ax, grid_size, rf_size, title, detail=''):
    """격자에서 수용 영역을 시각화"""
    ax.set_xlim(-0.5, grid_size - 0.5)
    ax.set_ylim(-0.5, grid_size - 0.5)
    ax.set_aspect('equal')
    # 격자
    for i in range(grid_size):
        for j in range(grid_size):
            color = '#e8f4f8'
            rect = plt.Rectangle((j-0.5, i-0.5), 1, 1, fill=True, facecolor=color, edgecolor='gray', lw=0.5)
            ax.add_patch(rect)
    # 수용 영역 강조
    offset = (grid_size - rf_size) // 2
    for i in range(rf_size):
        for j in range(rf_size):
            ri, rj = offset + i, offset + j
            rect = plt.Rectangle((rj-0.5, ri-0.5), 1, 1, fill=True, facecolor='#ff7f7f', edgecolor='darkred', lw=1.5)
            ax.add_patch(rect)
    # 중심 뉴런 표시
    cx = cy = grid_size // 2
    ax.plot(cx, cy, 'b*', ms=12, zorder=5)
    ax.set_title(f'{title}\nRF = {rf_size}×{rf_size}', fontsize=9)
    ax.set_xlabel(detail, fontsize=8)
    ax.set_xticks([]); ax.set_yticks([])
    ax.invert_yaxis()

draw_receptive(axes[0], 9, 1, '입력층', '각 뉴런 → 1×1 픽셀')
draw_receptive(axes[1], 9, 3, 'Conv(3×3) × 1', '1개 conv → RF=3×3')
draw_receptive(axes[2], 9, 5, 'Conv(3×3) × 2', '2개 conv → RF=5×5')
draw_receptive(axes[3], 9, 9, 'Conv(3×3) × 4', '4개 conv → RF=9×9')

plt.suptitle('수용 영역(Receptive Field): 3×3 합성곱을 쌓을수록 확대', fontsize=12)
plt.tight_layout()
plt.show()

print('=== 수용 영역 크기 (3×3 conv 반복) ===')
rf = 1
for k in range(1, 6):
    rf = rf + 2  # 각 3×3 conv마다 +2
    print(f'  {k}개 conv(3×3) 후: RF = {rf}×{rf}')

print()
print('📌 핵심: 3×3 필터를 쌓는 것이 큰 필터 하나보다 효율적!')
print('   - 3×3 × 2 = 파라미터 2×(3×3)=18개,  RF=5×5')
print('   - 5×5 × 1 = 파라미터 1×(5×5)=25개,  RF=5×5')
print('   → 적은 파라미터로 같은 수용 영역 달성 (VGG 핵심 아이디어)')

---
## Part 2. CNN 구현 — LeNet

### 2-1. MLP vs CNN 비교

| | MLP (Flatten) | CNN (LeNet) |
|---|---|---|
| **입력 처리** | 모든 픽셀을 1D로 펼침 | 2D 공간 구조 유지 |
| **공간 관계** | 무시 | 위치 불변성(translation invariance) 학습 |
| **파라미터** | 매우 많음 (3×32×32=3072 → ...) | 적음 (가중치 공유) |
| **성능** | 낮음 | 높음 |

### 2-2. LeNet-5 구조 (CIFAR-10 수정 버전)

```
입력 (3×32×32)
  │
Conv(3→6, k=5) → ReLU → MaxPool(2)   : (6×14×14)
  │
Conv(6→16, k=5) → ReLU → MaxPool(2)  : (16×5×5)
  │
Flatten → Linear(400→120) → ReLU
  │
Linear(120→84) → ReLU
  │
Linear(84→10)  → 출력 (10 클래스)
```

In [ ]:
# MLP Baseline & LeNet 클래스 정의

class MLP_CIFAR(nn.Module):
    """CIFAR-10 MLP baseline: 이미지를 1D로 펼쳐서 분류"""
    def __init__(self, hidden_dims=(512, 256), dropout=0.3):
        super().__init__()
        layers = [nn.Flatten()]
        in_dim = 3 * 32 * 32
        for h in hidden_dims:
            layers += [nn.Linear(in_dim, h), nn.ReLU(), nn.Dropout(dropout)]
            in_dim = h
        layers.append(nn.Linear(in_dim, 10))
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        return self.net(x)


class LeNet(nn.Module):
    """LeNet-5 (CIFAR-10 수정 버전): 3×32×32 입력"""
    def __init__(self):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 6, kernel_size=5),    # (3,32,32) → (6,28,28)
            nn.ReLU(),
            nn.MaxPool2d(2, 2),                # (6,28,28) → (6,14,14)
            nn.Conv2d(6, 16, kernel_size=5),   # (6,14,14) → (16,10,10)
            nn.ReLU(),
            nn.MaxPool2d(2, 2),                # (16,10,10) → (16,5,5)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(16 * 5 * 5, 120),
            nn.ReLU(),
            nn.Linear(120, 84),
            nn.ReLU(),
            nn.Linear(84, 10),
        )

    def forward(self, x):
        return self.classifier(self.features(x))


def count_params(model):
    return sum(p.numel() for p in model.parameters())


# 구조 확인
mlp   = MLP_CIFAR()
lenet = LeNet()

print('=== MLP_CIFAR 구조 ===')
print(mlp)
print(f'\n총 파라미터: {count_params(mlp):,}')

print('\n=== LeNet 구조 ===')
print(lenet)
print(f'\n총 파라미터: {count_params(lenet):,}')

# 순전파 크기 확인
with torch.no_grad():
    dummy = torch.randn(1, 3, 32, 32)
    print(f'\n입력:        {dummy.shape}')
    print(f'conv1 출력:  {lenet.features[:2](dummy).shape}')
    print(f'pool1 출력:  {lenet.features[:3](dummy).shape}')
    print(f'conv2 출력:  {lenet.features[:5](dummy).shape}')
    print(f'pool2 출력:  {lenet.features(dummy).shape}')
    print(f'최종 출력:   {lenet(dummy).shape}')

In [ ]:
# 공유 학습 함수

def train_cnn(model, train_loader, test_loader,
              n_epochs=15, lr=0.001, verbose=True):
    """CNN/MLP 학습 함수 (디바이스 자동 처리)"""
    model = model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()

    train_losses, val_losses, train_accs, val_accs = [], [], [], []

    for epoch in range(n_epochs):
        # --- 학습 ---
        model.train()
        ep_loss, correct, total = 0., 0, 0
        for X, y in train_loader:
            X, y = X.to(device), y.to(device)
            optimizer.zero_grad()
            logits = model(X)
            loss = criterion(logits, y)
            loss.backward()
            optimizer.step()
            ep_loss  += loss.item() * len(X)
            correct  += (logits.argmax(1) == y).sum().item()
            total    += len(y)

        train_losses.append(ep_loss / total)
        train_accs.append(correct / total)

        # --- 검증 ---
        model.eval()
        v_loss, v_correct, v_total = 0., 0, 0
        with torch.no_grad():
            for X, y in test_loader:
                X, y = X.to(device), y.to(device)
                logits = model(X)
                v_loss   += criterion(logits, y).item() * len(X)
                v_correct += (logits.argmax(1) == y).sum().item()
                v_total   += len(y)

        val_losses.append(v_loss / v_total)
        val_accs.append(v_correct / v_total)

        if verbose:
            print(f'Epoch {epoch+1:2d}/{n_epochs} | '
                  f'Train {train_losses[-1]:.4f}/{train_accs[-1]:.4f} | '
                  f'Val {val_losses[-1]:.4f}/{val_accs[-1]:.4f}')

    model.cpu()
    return train_losses, val_losses, train_accs, val_accs


print('train_cnn() 함수 준비 완료!')
print(f'학습 디바이스: {device}')

In [ ]:
# MLP vs LeNet 학습 및 비교 (n_epochs=15)
print('=== MLP_CIFAR 학습 ===')
torch.manual_seed(42)
model_mlp = MLP_CIFAR()
mlp_tl, mlp_vl, mlp_ta, mlp_va = train_cnn(model_mlp, train_loader, test_loader, n_epochs=15)

print('\n=== LeNet 학습 ===')
torch.manual_seed(42)
model_lenet = LeNet()
ln_tl, ln_vl, ln_ta, ln_va = train_cnn(model_lenet, train_loader, test_loader, n_epochs=15)

# 비교 시각화
ep = range(1, 16)
fig, axes = plt.subplots(1, 2, figsize=(13, 4))

for (name, vl, va, color) in [
    (f'MLP  (params={count_params(model_mlp):,})',   mlp_vl, mlp_va, 'steelblue'),
    (f'LeNet(params={count_params(model_lenet):,})', ln_vl,  ln_va,  'tomato'),
]:
    axes[0].plot(ep, vl, color=color, lw=2, label=name)
    axes[1].plot(ep, va, color=color, lw=2, label=f'{name} ({va[-1]:.4f})')

for ax, title, ylabel in [
    (axes[0], 'MLP vs LeNet: Val Loss',     'CrossEntropy Loss'),
    (axes[1], 'MLP vs LeNet: Val Accuracy', 'Accuracy'),
]:
    ax.set_title(title); ax.set_xlabel('에포크')
    ax.set_ylabel(ylabel); ax.legend(fontsize=9)

plt.suptitle('MLP vs LeNet — CIFAR-10 분류 (10,000 학습 샘플, 15 에포크)', fontsize=12)
plt.tight_layout()
plt.show()

print(f'\n최종 Val Accuracy')
print(f'  MLP  : {mlp_va[-1]:.4f}  ({count_params(model_mlp):,} params)')
print(f'  LeNet: {ln_va[-1]:.4f}  ({count_params(model_lenet):,} params)')
print(f'  → LeNet이 {count_params(model_mlp)//count_params(model_lenet)}배 적은 파라미터로 더 높은 성능!')

In [ ]:
# 학습된 필터 & Feature Map 시각화

# --- Conv1 필터 시각화 (학습된 6개 필터) ---
conv1_weights = model_lenet.features[0].weight.data  # (6, 3, 5, 5)
fig, axes = plt.subplots(1, 6, figsize=(12, 2.5))
for i in range(6):
    # 3채널 필터를 RGB로 시각화
    filt = conv1_weights[i].permute(1, 2, 0).numpy()  # (5,5,3)
    filt = (filt - filt.min()) / (filt.max() - filt.min() + 1e-8)
    axes[i].imshow(filt)
    axes[i].set_title(f'필터 {i+1}', fontsize=9)
    axes[i].axis('off')
plt.suptitle('LeNet Conv1 학습된 필터 (6개, 5×5×3)', fontsize=11)
plt.tight_layout()
plt.show()

# --- Feature Map 시각화 (특정 이미지) ---
activations = {}

def make_hook(name):
    def hook(module, inp, out):
        activations[name] = out.detach()
    return hook

h1 = model_lenet.features[0].register_forward_hook(make_hook('conv1'))  # Conv2d
h2 = model_lenet.features[3].register_forward_hook(make_hook('conv2'))  # Conv2d

# 테스트 이미지 한 장 forward
sample_img, sample_lbl = testset_full[7]
with torch.no_grad():
    _ = model_lenet(sample_img.unsqueeze(0))

h1.remove(); h2.remove()

fig, axes = plt.subplots(3, 8, figsize=(15, 6))

# 원본 이미지
orig = denorm(sample_img).permute(1, 2, 0).numpy()
axes[0, 0].imshow(orig)
axes[0, 0].set_title(f'원본\n({classes[sample_lbl]})', fontsize=8)
for j in range(1, 8): axes[0, j].axis('off')

# Conv1 feature maps (6개)
fmaps1 = activations['conv1'][0]  # (6, 28, 28)
for i in range(6):
    axes[1, i].imshow(fmaps1[i].numpy(), cmap='viridis')
    axes[1, i].set_title(f'Conv1[{i+1}]\n28×28', fontsize=8)
    axes[1, i].axis('off')
for j in range(6, 8): axes[1, j].axis('off')

# Conv2 feature maps (8개 중 처음 8개)
fmaps2 = activations['conv2'][0]  # (16, 10, 10)
for i in range(8):
    axes[2, i].imshow(fmaps2[i].numpy(), cmap='viridis')
    axes[2, i].set_title(f'Conv2[{i+1}]\n10×10', fontsize=8)
    axes[2, i].axis('off')

axes[0, 0].axis('on'); axes[0, 0].set_xticks([]); axes[0, 0].set_yticks([])
axes[0, 0].set_ylabel('입력', fontsize=9)
axes[1, 0].set_ylabel('Conv1\n(6ch)', fontsize=9)
axes[2, 0].set_ylabel('Conv2\n(16ch)', fontsize=9)

plt.suptitle(f'LeNet Feature Maps — {classes[sample_lbl]}', fontsize=12)
plt.tight_layout()
plt.show()

print('📌 관찰: 초기 레이어는 edge/texture를, 깊은 레이어는 고수준 패턴을 감지합니다.')

---
## Part 3. 더 깊은 CNN — VGG-style

### 3-1. VGGNet 핵심 아이디어

2014년 Oxford VGG팀이 제안한 VGGNet의 핵심은 **3×3 합성곱만 반복 사용**입니다.

| 특징 | 내용 |
|---|---|
| 모든 conv | 3×3, padding=1 (same) |
| 다운샘플링 | MaxPool(2×2)만 사용 |
| 채널 수 | 블록마다 2배 (32→64→128) |
| 정규화 | BatchNorm + Dropout |

### 3-2. VGG-style 블록 구조

```
Block 1: Conv(3→32)→BN→ReLU → Conv(32→32)→BN→ReLU → MaxPool → Dropout2d(0.1)
         입력 32×32 → 출력 16×16

Block 2: Conv(32→64)→BN→ReLU → Conv(64→64)→BN→ReLU → MaxPool → Dropout2d(0.2)
         입력 16×16 → 출력 8×8

Block 3: Conv(64→128)→BN→ReLU → Conv(128→128)→BN→ReLU → MaxPool
         입력 8×8 → 출력 4×4

Classifier: Flatten(128×4×4=2048) → Linear(256) → ReLU → Dropout → Linear(10)
```

In [ ]:
# VGG-style CNN 클래스

def vgg_block(in_ch, out_ch, dropout2d=0.0):
    """VGG-style 블록: Conv×2 + BN + ReLU + MaxPool (+ 선택적 Dropout2d)"""
    layers = [
        nn.Conv2d(in_ch,  out_ch, 3, padding=1), nn.BatchNorm2d(out_ch), nn.ReLU(),
        nn.Conv2d(out_ch, out_ch, 3, padding=1), nn.BatchNorm2d(out_ch), nn.ReLU(),
        nn.MaxPool2d(2, 2),
    ]
    if dropout2d > 0:
        layers.append(nn.Dropout2d(dropout2d))
    return nn.Sequential(*layers)


class VGGNet(nn.Module):
    """소형 VGG-style CNN for CIFAR-10"""
    def __init__(self, dropout=0.5):
        super().__init__()
        self.features = nn.Sequential(
            vgg_block(3,   32,  dropout2d=0.1),   # 32×32 → 16×16
            vgg_block(32,  64,  dropout2d=0.2),   # 16×16 →  8×8
            vgg_block(64,  128, dropout2d=0.0),   #  8×8  →  4×4
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128 * 4 * 4, 256),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(256, 10),
        )

    def forward(self, x):
        return self.classifier(self.features(x))


# 구조 확인
vgg = VGGNet()
print('=== VGGNet 구조 ===')
print(vgg)
print(f'\n총 파라미터: {count_params(vgg):,}')

with torch.no_grad():
    dummy = torch.randn(1, 3, 32, 32)
    for i, block in enumerate(vgg.features):
        dummy = block(dummy)
        print(f'Block {i+1} 출력: {dummy.shape}')
    print(f'Flatten: {dummy.view(1,-1).shape}')

In [ ]:
# LeNet vs VGGNet 비교 학습
print('=== VGGNet 학습 ===')
torch.manual_seed(42)
model_vgg = VGGNet()
vgg_tl, vgg_vl, vgg_ta, vgg_va = train_cnn(model_vgg, train_loader, test_loader, n_epochs=15)

# 3개 모델 비교 시각화
ep = range(1, 16)
results = [
    ('MLP',    mlp_va,  mlp_vl,  'steelblue', count_params(model_mlp)),
    ('LeNet',  ln_va,   ln_vl,   'tomato',    count_params(model_lenet)),
    ('VGGNet', vgg_va,  vgg_vl,  'seagreen',  count_params(model_vgg)),
]

fig, axes = plt.subplots(1, 2, figsize=(13, 5))
for name, va, vl, color, _ in results:
    axes[0].plot(ep, vl, color=color, lw=2, label=name)
    axes[1].plot(ep, va, color=color, lw=2, label=f'{name} ({va[-1]:.4f})')

for ax, title, ylabel in [
    (axes[0], 'Val Loss 비교',     'Loss'),
    (axes[1], 'Val Accuracy 비교', 'Accuracy'),
]:
    ax.set_title(title); ax.set_xlabel('에포크')
    ax.set_ylabel(ylabel); ax.legend(fontsize=9)

plt.suptitle('MLP vs LeNet vs VGGNet — CIFAR-10 (15 에포크)', fontsize=12)
plt.tight_layout()
plt.show()

# 성능 표
print(f'\n{"모델":<10} {"Val Acc":>10} {"파라미터":>12}')
print('-' * 36)
for name, va, _, _, n_p in results:
    mark = ' ★' if va[-1] == max(r[1][-1] for r in results) else ''
    print(f'{name:<10} {va[-1]:>10.4f} {n_p:>12,}{mark}')

---
## Exercise

### Exercise 1. LeNet_wide — 채널 수 확장 실험

기준 LeNet의 필터 채널 수를 **2배**로 늘린 `LeNet_wide`를 구현하고 비교하세요.

| 레이어 | LeNet (기준) | LeNet_wide |
|---|---|---|
| Conv1 | 3 → 6 | 3 → 12 |
| Conv2 | 6 → 16 | 12 → 32 |
| FC1 | 400 → 120 | **32×5×5** → 240 |
| FC2 | 120 → 84 | 240 → 168 |
| FC3 | 84 → 10 | 168 → 10 |

**요구사항:**
1. `LeNet_wide` 클래스 구현 (LeNet과 동일한 구조, 채널 수만 2배)
2. 동일 조건으로 학습 (n_epochs=15, Adam, lr=0.001)
3. Val Accuracy 곡선 비교 + 파라미터 수 및 최종 Val Acc 출력

In [ ]:
# Exercise 1: LeNet_wide 구현

class LeNet_wide(nn.Module):
    """LeNet 채널 수 2배 확장 버전"""
    def __init__(self):
        super().__init__()
        # Your code here: features 정의
        #   Conv(3→12, k=5) → ReLU → MaxPool(2)
        #   Conv(12→32, k=5) → ReLU → MaxPool(2)
        self.features = None

        # Your code here: classifier 정의
        #   Flatten → Linear(32*5*5, 240) → ReLU
        #   Linear(240, 168) → ReLU
        #   Linear(168, 10)
        self.classifier = None

    def forward(self, x):
        # Your code here
        pass


# Your code here: LeNet_wide 학습 (n_epochs=15, lr=0.001)
torch.manual_seed(42)
model_lenet_wide = LeNet_wide()

# lw_tl, lw_vl, lw_ta, lw_va = train_cnn(model_lenet_wide, train_loader, test_loader, n_epochs=15)

# Your code here: LeNet vs LeNet_wide Val Accuracy 곡선 비교

# Your code here: 파라미터 수 및 최종 Val Acc 출력

### Exercise 2. Feature Map 시각화

학습된 **VGGNet**의 각 Block 출력 feature map을 추출하고 시각화하세요.

**요구사항:**
1. `register_forward_hook`을 사용해 Block 1, 2, 3 출력 저장
2. 테스트셋에서 이미지 1장 선택 (임의 선택 가능)
3. 각 블록의 feature map을 채널 최대 8개 시각화 (총 3행×8열)
4. 블록이 깊어질수록 feature map 크기와 내용이 어떻게 변하는지 관찰

**힌트:**
```python
handle = model.features[0].register_forward_hook(make_hook('block1'))
# ... forward pass ...
handle.remove()  # hook 해제!
```

In [ ]:
# Exercise 2: VGGNet Feature Map 시각화

# Your code here: 테스트 이미지 선택
# sample_img, sample_lbl = testset_full[?]

# Your code here: Block 1, 2, 3에 hook 등록
# h1 = model_vgg.features[0].register_forward_hook(...)
# h2 = ...
# h3 = ...

# Your code here: forward pass 실행
# model_vgg.eval()
# with torch.no_grad():
#     _ = model_vgg(sample_img.unsqueeze(0))

# Your code here: hook 해제

# Your code here: 3행×8열 feature map 시각화
# - 행: Block 1 / Block 2 / Block 3
# - 열: 채널 0~7 (최대 8개)
# - 각 subplot에 크기 정보 표시 (예: '16×16')

### Exercise 3. (도전) Global Average Pooling (GAP)

VGGNet의 대형 FC 분류기 대신 **Global Average Pooling**으로 파라미터를 크게 줄이세요.

**GAP 작동 방식:**
```
일반 Flatten:  (B, 128, 4, 4) → (B, 2048)  → Linear(2048→256) → ...
GAP:           (B, 128, 4, 4) → GAP → (B, 128) → Linear(128→10)  # FC 1개만!
```

**요구사항:**
1. `GAP_CNN` 클래스 구현 (`nn.AdaptiveAvgPool2d(1)` 사용)
   - `features`는 VGGNet과 동일
   - `classifier`는 GAP → Flatten → Linear(128, 10) 만
2. 동일 조건으로 학습 (n_epochs=15)
3. VGGNet vs GAP_CNN: 파라미터 수 비교 및 Val Accuracy 비교

**힌트:**
```python
self.gap = nn.AdaptiveAvgPool2d(1)  # (B,C,H,W) → (B,C,1,1)
# forward에서: x = self.gap(x).squeeze(-1).squeeze(-1)  → (B,C)
```

In [ ]:
# Exercise 3: Global Average Pooling CNN

class GAP_CNN(nn.Module):
    """Global Average Pooling을 사용하는 CNN"""
    def __init__(self):
        super().__init__()
        # Your code here: features (VGGNet과 동일한 3개 블록)
        self.features = None

        # Your code here: GAP layer (AdaptiveAvgPool2d(1))
        self.gap = None

        # Your code here: classifier (Linear(128, 10)만)
        self.classifier = None

    def forward(self, x):
        # Your code here:
        # 1. features 통과
        # 2. GAP 적용
        # 3. squeeze
        # 4. classifier 적용
        pass


# Your code here: GAP_CNN 학습 (n_epochs=15)
torch.manual_seed(42)
model_gap = GAP_CNN()

# gap_tl, gap_vl, gap_ta, gap_va = train_cnn(model_gap, train_loader, test_loader, n_epochs=15)

# Your code here: VGGNet vs GAP_CNN
# - 파라미터 수 비교
# - Val Accuracy 곡선 비교
# - 결과 해석: 적은 파라미터로 비슷한 성능이 나오는가?

---
## Summary

| 개념 | 핵심 내용 |
|---|---|
| **합성곱 연산** | 필터가 이미지 위를 슬라이딩하며 내적 계산 |
| **특성맵** | 각 필터마다 하나씩 생성되는 출력 |
| **수용 영역** | 각 출력 뉴런이 참조하는 입력 영역; 레이어 증가 시 확장 |
| **스트라이드** | 필터 이동 보폭; S=2이면 출력 크기 ½ |
| **패딩** | 가장자리 0 추가; same padding으로 크기 유지 |
| **MaxPooling** | 지역 최댓값 추출; 공간 다운샘플링 + 위치 불변성 |
| **LeNet** | Conv×2 + MaxPool×2 + FC×3; 초기 CNN 기준 모델 |
| **VGGNet** | 3×3 conv 반복 + MaxPool; 3×3×2 = 5×5 동일 RF, 적은 파라미터 |
| **BatchNorm (CNN)** | BatchNorm2d; 채널별 정규화, 학습 안정성 향상 |
| **가중치 공유** | 같은 필터를 이미지 전체에 공유 → 파라미터 효율 |
| **GAP** | FC 분류기 대신 채널별 평균 → 파라미터 대폭 감소 |

---

**다음 강의 (Week 13):** RNN — 순환 신경망, LSTM, 시계열 데이터